## Import

In [1]:
from data_aug.data_aug import *
from data_aug.bbox_util import *
import numpy as np 
import cv2 
import matplotlib.pyplot as plt 
import pickle as pkl
%matplotlib inline

## Helper classes

In [56]:
class AugGenerator:
    """How to use: set_input_dir(dir) -> read_input() -> set_sequences(sequences) -> set_default_resolution((w,h))-> execute()"""
    def to_yolov3_format(self, bboxes):
        _bboxes = []
        w_img,h_img = self.w, self.h
        for bbox in bboxes:
            w_box, h_box = bbox[2]-bbox[0], bbox[3]-bbox[1]
            _bbox = [int(bbox[4]),bbox[0]+w_box/2,bbox[1]+h_box/2,w_box,h_box]
            _bbox = [_bbox[0],_bbox[1]/w_img,_bbox[2]/h_img,_bbox[3]/w_img,_bbox[4]/h_img]
            _bboxes.append(_bbox)
        return _bboxes
    def revert_format(self, bboxes):
        _bboxes = []
        w_img,h_img = self.w, self.h
        for bbox in bboxes:
            _bbox = [bbox[1]*w_img,bbox[2]*h_img,bbox[3]*w_img,bbox[4]*h_img,bbox[0]]
            w_box, h_box = _bbox[2], _bbox[3]
            _bbox = [_bbox[0]-w_box/2, _bbox[1]-h_box/2,_bbox[0]+w_box/2, _bbox[1]+h_box/2, _bbox[4]]
            _bboxes.append(_bbox)
        return _bboxes
    def set_input(self, imgPaths, labelPaths):
        self.imgPaths = imgPaths
        self.labelPaths = labelPaths
        return self
    def read_input(self):
        self.labels = []
        for path in self.labelPaths:
            with open(path,'r') as txt:
                bboxes = []
                for line in txt.readlines():
                    bboxes.append([int(i) for i in line.split(' ')])                  
                bboxes = np.array(bboxes,dtype='float')
            self.labels.append(bboxes)
        return self
    def set_default_resolution(self, size):
        self.w, self.h = size[0],size[1]
        return self
    def set_sequences(self, sequences):
        self.sequences = sequences
        return self
    def get_default_sequences(self):
        sequences = [Rotate(0),Rotate(45),Rotate(-45),RandomHorizontalFlip(1)]
        tmp = []
        for seq in sequences:
            tmp = tmp + [Sequence([seq,RandomHSV((50,50),0,(-10,-10))])]
        sequences = sequences+tmp
        return sequences
    def execute(self, outDir):
        n = len(self.imgPaths)
        for i in range(n):
            path = self.imgPaths[i]
            img = cv2.imread(path)
            bboxes = self.labels[i]
            indx = path.index('/')
            name = path[indx+1:path.index('.')]
            print(i,n,path)

            for j in range(len(self.sequences)):
                # do transformation
                augImg, augBboxes = self.sequences[j](img.copy(),np.array(bboxes))
                # transform to yolov3 format
                augBboxes = self.to_yolov3_format(augBboxes)
                # save transformed image+'aug%03d'%j
                cv2.imwrite('%s/%saug%03d.jpg'%(outDir,name,j),augImg)
                # save transformed bbox to label file
                with open('%s/%saug%03d.txt'%(outDir,name,j),'w', newline='\n') as txt:
                    for bbox in augBboxes:
                        line = ' '.join([str(item) for item in bbox])+'\n'
                        txt.write(line)
        return self              



In [3]:
class TrainTestSpliter:
    """Use this class to quickly split the output of the AugGenerator input train/test subfolders.
        How to use: set_input(dir) -> split()"""
    def set_input(self, dir):
        self.imgPaths = sorted(glob.glob('%s/%s'%(dir,'*.jpg')))
        self.labelPaths = sorted(glob.glob('%s/%s'%(dir,'*.txt')))
        self.dir = dir
        return self
    def split(self, trainRatio=.8):
        from shutil import move
        from os import mkdir
        import numpy as np
        
        dir_train, dir_test = self.dir+'/train', self.dir+'/test'
        try: os.mkdir(dir_train)
        except: pass
        try: os.mkdir(dir_test)
        except: pass
        n = len(self.imgPaths)
        nums = int(n*trainRatio)
        indices = np.linspace(0,n-1,num=nums,dtype='int').tolist()
        # print(nums,n)
        # print(indices)
        for i in range(n):
            src_img, src_txt = self.imgPaths[i], self.labelPaths[i]
            name = src_img[src_img.rindex('/')+1:src_img.rindex('.')]
            dir_dest = None
            try:
                indices.index(i) # checks if i in indices list
                dir_dest = dir_train
            except:
                dir_dest = dir_test
            # print(i,src_img,'%s/%s.jpg'%(dir_dest,name))
            move(src_img,'%s/%s.jpg'%(dir_dest,name))
            move(src_txt,'%s/%s.txt'%(dir_dest,name))


In [45]:
class AugValidator:
    def set_default_resolution(self, sz):
        self.w, self.h = sz[0],sz[1]
        return self
    def set_input(self, imgPaths, labelPaths):
        self.imgPaths, self.labelPaths = imgPaths, labelPaths
        return self
    def read_input(self):
        self.labels = []
        for path in self.labelPaths:
            with open(path,'r') as txt:
                bboxes = []
                for line in txt.readlines():
                    nums = line.split(' ')
                    bboxes.append([int(nums[0])]+[float(x) for x in nums[1:]])                  
            self.labels.append(bboxes)
        return self
    def revert_format(self):
        self.rlabels = []
        w_img,h_img = self.w, self.h
        for bboxes in self.labels:
            rbboxes = []
            for bbox in bboxes:
                _bbox = [bbox[1]*w_img,bbox[2]*h_img,bbox[3]*w_img,bbox[4]*h_img,bbox[0]]
                w_box, h_box = _bbox[2], _bbox[3]
                _bbox = [_bbox[0]-w_box/2, _bbox[1]-h_box/2,_bbox[0]+w_box/2, _bbox[1]+h_box/2, _bbox[4]]
                rbboxes.append(_bbox)
            rbboxes = np.array(rbboxes,dtype='int')
            self.rlabels.append(rbboxes)
        return self

# Augmenting

In [57]:
import glob
sz = (640,360)  # sets dimensions to resize images
input = [sorted(glob.glob('input/*.jpg')), sorted(glob.glob('input/*.txt'))]    # gets all image and label files
g = AugGenerator()
g.set_input(input[0],input[1]).read_input().set_default_resolution(sz).set_sequences(g.get_default_sequences()).execute('output-test')  # chaining methods to augments images


45
-45
194 586 input/tung3100000743.jpg
0
45
-45
0
45
-45
195 586 input/tung3100000757.jpg
0
45
-45
0
45
-45
196 586 input/tung3100000770.jpg
0
45
-45
0
45
-45
197 586 input/tung3100000780.jpg
0
45
-45
0
45
-45
198 586 input/tung3100000788.jpg
0
45
-45
0
45
-45
199 586 input/tung3100000792.jpg
0
45
-45
0
45
-45
200 586 input/tung3100000799.jpg
0
45
-45
0
45
-45
201 586 input/tung3100000817.jpg
0
45
-45
0
45
-45
202 586 input/tung3100000836.jpg
0
45
-45
0
45
-45
203 586 input/tung3100000846.jpg
0
45
-45
0
45
-45
204 586 input/tung3100000853.jpg
0
45
-45
0
45
-45
205 586 input/tung3100000861.jpg
0
45
-45
0
45
-45
206 586 input/tung3100000870.jpg
0
45
-45
0
45
-45
207 586 input/tung3100000875.jpg
0
45
-45
0
45
-45
208 586 input/tung3100000880.jpg
0
45
-45
0
45
-45
209 586 input/tung3100000882.jpg
0
45
-45
0
45
-45
210 586 input/tung3100000886.jpg
0
45
-45
0
45
-45
211 586 input/tung3100000888.jpg
0
45
-45
0
45
-45
212 586 input/tung3100000892.jpg
0
45
-45
0
45
-45
213 586 input/tung31000

# Split to train/test datasets

In [58]:
spt = TrainTestSpliter().set_input('output-test').split()

In [60]:
a = len(os.listdir('output-test/train'))
b = len(os.listdir('output-test/test'))
print('train',a,'test',b,a+b)

train 7500 test 1876 9376


In [8]:
%%bash
mv output members-dataset-vinh
zip -r members-dataset-vinh.zip members-dataset-vinh

%)
  adding: members-dataset-vinh/train/vinh-200000378aug014.jpg (deflated 3%)
  adding: members-dataset-vinh/train/vinh-200000210aug023.txt (deflated 50%)
  adding: members-dataset-vinh/train/vinh-200000387aug006.jpg (deflated 0%)
  adding: members-dataset-vinh/train/vinh00000325aug015.txt (deflated 55%)
  adding: members-dataset-vinh/train/vinh-200000396aug006.txt (deflated 47%)
  adding: members-dataset-vinh/train/vinh-200000378aug021.jpg (deflated 0%)
  adding: members-dataset-vinh/train/vinh00000590aug022.jpg (deflated 3%)
  adding: members-dataset-vinh/train/vinh-200000267aug003.jpg (deflated 3%)
  adding: members-dataset-vinh/train/vinh-200000243aug021.jpg (deflated 0%)
  adding: members-dataset-vinh/train/vinh-200000378aug021.txt (deflated 50%)
  adding: members-dataset-vinh/train/vinh00000610aug013.jpg (deflated 7%)
  adding: members-dataset-vinh/train/vinh00000315aug009.txt (deflated 54%)
  adding: members-dataset-vinh/train/vinh-200000405aug001.jpg (deflated 3%)
  adding: me